In [1]:
# # Install needed libraries
# !python -m pip install import_ipynb 

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from crunch_func import *

In [2]:
try:
    engine_df = pd.read_csv("EngineData.txt", sep='\t')
except FileNotFoundError:
        print("Engine Data file found. Make sure file is in same directory and name is correct.")


In [3]:
cd1700 = load_data("clean_1700.csv",'1700_001.txt')
cd1700 = clean_data(cd1700)

Found clean_1700.csv, CSV converted into dataframe.


In [4]:
print("*** DATA SET 1700 ***")
print("Number of Cycles: ", max(cd1700['cycle']))
print("Highest Pressure Reached: ", max(cd1700['pressure_bar']))
print("Average Pressure: ", sum(cd1700['pressure_bar'])/len(cd1700['pressure_bar']))

eng1_data = engine_df.iloc[0]
load = eng1_data["Load"]
espeed = eng1_data["Engine Speed (rpm)"]
torque = eng1_data["Torque (lb-f)"]
power = eng1_data["Power (hp)"]
time_2_leave = eng1_data["Time for 0.488 L of fuel to leave Burette (s)"]
air_flow_rate = eng1_data["Air Flow Rate (CFM)"]
air_in_temp = eng1_data["Air Intake Temperature (F)"]

# Select only wanted columns
# cd1700 = cd1700[['time','cycle','stage','crank_ang','pressure_bar']]  
# Select by cycle number
# df = select_cycles(cd1700, [5,6,7])

# Select which cycles need to be plotted
# plt.scatter(df['crank_ang'],df['pressure_bar'], marker='.')
# plt.xlim((-50,1000))

*** DATA SET 1700 ***
Number of Cycles:  2972
Highest Pressure Reached:  31.771551599999995
Average Pressure:  5.2649481825899995


In [5]:
print("*** DATA SET 2730 ***")
cd2730 = load_data("clean_2730.csv",'2750_001.txt')
print("Number of Cycles: ", max(cd2730['cycle']))
print("Highest Pressure Reached: ", max(cd2730['pressure_bar']))
print("Average Pressure: ", sum(cd2730['pressure_bar'])/len(cd2730['pressure_bar']))
cd2730 = clean_data(cd2730)

*** DATA SET 2730 ***
Found clean_2730.csv, CSV converted into dataframe.
Number of Cycles:  5906
Highest Pressure Reached:  38.63614139999999
Average Pressure:  1.59826272276


In [ ]:
plt.plot(cd2730['cycle'], cd2730['ref_p'])

In [ ]:
eng2_data = engine_df.iloc[1]
load = eng2_data["Load"]
espeed = eng2_data["Engine Speed (rpm)"]
torque = eng2_data["Torque (lb-f)"]
power = eng2_data["Power (hp)"]
time_2_leave = eng2_data["Time for 0.488 L of fuel to leave Burette (s)"]
air_flow_rate = eng2_data["Air Flow Rate (CFM)"]
air_in_temp = eng2_data["Air Intake Temperature (F)"]

# Select only wanted columns
# cd2730 = cd2730[['time','cycle','stage','crank_ang','pressure_bar','ref_p']]
print(cd2730.head())

# Select by cycle number
df = select_cycles(cd2730, list(range(1000)))

# Select which cycles need to be plotted
# plt.scatter(df['crank_ang'],df['pressure_bar'], marker='.')
# plt.xlim((-50,1000))

In [ ]:
print("*** DATA SET cd4500 ***")
cd4500 = load_data("clean_4500.csv",'4500_001.txt')
print("Number of Cycles: ", max(cd4500['cycle']))
print("Highest Pressure Reached: ", max(cd4500['pressure_bar']))
print("Average Pressure: ", sum(cd4500['pressure_bar'])/len(cd4500['pressure_bar']))

In [ ]:


eng3_data = engine_df.iloc[2]
load = eng3_data["Load"]
espeed = eng3_data["Engine Speed (rpm)"]
torque = eng3_data["Torque (lb-f)"]
power = eng3_data["Power (hp)"]
time_2_leave = eng3_data["Time for 0.488 L of fuel to leave Burette (s)"]
air_flow_rate = eng3_data["Air Flow Rate (CFM)"]
air_in_temp = eng3_data["Air Intake Temperature (F)"]

# Select only wanted columns
cd4500 = cd4500[['cycle','stage','crank_ang','pressure_bar']]
print(cd4500.head())

# Select by cycle number
df = select_cycles(cd4500, list(range(1002, 1003)))

# Select which cycles need to be plotted
plt.scatter(df['crank_ang'],df['pressure_bar'], marker='.')
plt.xlim((-50,1000))